## Описание проекта

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

# 1. Подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from catboost import CatBoostRegressor
import lightgbm as lgb
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/datasets/autos.csv')

In [3]:
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


In [5]:
df.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [6]:
# Удалим столбцы, которые, скорее всего, не имеют силы в предсказании
df = df.drop(columns = ['DateCrawled', 'DateCreated', 'LastSeen', 'NumberOfPictures'])

In [7]:
# В данном столбце можно предположить, что если данные отсутствуют, значит ремонтных работ не проводилось, поэтому сделаем замену
for i in ['NotRepaired', 'VehicleType', 'Model', 'FuelType', 'Gearbox']:
    df[i] = df[i].fillna(0)
    df[i] = df[i].replace(0, 'unspecified')

df.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,PostalCode
0,480,unspecified,1993,manual,0,golf,150000,0,petrol,volkswagen,unspecified,70435
1,18300,coupe,2011,manual,190,unspecified,125000,5,gasoline,audi,yes,66954
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,unspecified,90480
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,91074
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,60437


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 12 columns):
Price                354369 non-null int64
VehicleType          354369 non-null object
RegistrationYear     354369 non-null int64
Gearbox              354369 non-null object
Power                354369 non-null int64
Model                354369 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             354369 non-null object
Brand                354369 non-null object
NotRepaired          354369 non-null object
PostalCode           354369 non-null int64
dtypes: int64(6), object(6)
memory usage: 32.4+ MB


In [9]:
# Перекодируем в численные значения
df = df.apply(LabelEncoder().fit_transform)

In [10]:
# Прямое кодирование
df = pd.get_dummies(df, columns = ['NotRepaired', 'Gearbox'], drop_first = True)

In [11]:
# Разбивка датасета на тренировочную, валидационную и тестовую выборки
df_train, df_test = train_test_split(df, test_size=0.2, random_state=12345)
df_train, df_valid = train_test_split(df_train, test_size=0.25, random_state=12345)
features_train = df_train.drop(['Price'], axis=1)
target_train = df_train['Price']
features_valid = df_valid.drop(['Price'], axis=1)
target_valid = df_valid['Price']
features_test = df_test.drop(['Price'], axis=1)
target_test = df_test['Price']

# 2. Обучение моделей

### CatBoost

In [12]:
model_cb = CatBoostRegressor(random_state = 12345, iterations = 300)
parameters_cb = {'depth': range(6, 11),
                 'learning_rate': [0.001, 0.01, 0.05, 0.1]}
search_cb = RandomizedSearchCV(model_cb, parameters_cb, cv = 5, n_jobs = -1, random_state = 12345)
search_cb.fit(features_train, target_train, verbose = 100)
best_cb = search_cb.best_estimator_
predict_cb_valid = best_cb.predict(features_valid)
rmse_cb_valid = mean_squared_error(target_valid, predict_cb_valid)**0.5
print('Качество модели CatBoost на валидационной выборке (корень из средней квадратичной ошибки):', rmse_cb_valid)

0:	learn: 956.3999854	total: 216ms	remaining: 1m 4s
100:	learn: 585.2447044	total: 19.9s	remaining: 39.1s
200:	learn: 481.4648103	total: 39.1s	remaining: 19.3s
299:	learn: 446.0565172	total: 58.2s	remaining: 0us
0:	learn: 956.0195231	total: 173ms	remaining: 51.8s
100:	learn: 585.4519262	total: 20.1s	remaining: 39.6s
200:	learn: 481.9085961	total: 39.8s	remaining: 19.6s
299:	learn: 446.6376610	total: 58.2s	remaining: 0us
0:	learn: 956.9826825	total: 68.2ms	remaining: 20.4s
100:	learn: 585.7202549	total: 19.4s	remaining: 38.2s
200:	learn: 481.4428473	total: 38s	remaining: 18.7s
299:	learn: 446.2186321	total: 57.8s	remaining: 0us
0:	learn: 957.1410409	total: 143ms	remaining: 42.8s
100:	learn: 585.0717375	total: 19.1s	remaining: 37.6s
200:	learn: 480.3307249	total: 37.7s	remaining: 18.6s
299:	learn: 444.7134037	total: 57.4s	remaining: 0us
0:	learn: 956.0642499	total: 89ms	remaining: 26.6s
100:	learn: 585.6328271	total: 18.9s	remaining: 37.3s
200:	learn: 481.5039573	total: 37.5s	remaining: 

### LightGBM

In [13]:
model_lgb = lgb.LGBMRegressor(random_state=12345)
parameters_lgb = {'depth': range(6, 11),
                 'learning_rate': [0.001, 0.01, 0.05, 0.1],
                 'n_estimators': range(100, 501, 50)}
search_lgb = RandomizedSearchCV(model_lgb, parameters_lgb, cv = 5, n_jobs = -1, random_state = 12345)
search_lgb.fit(features_train, target_train)
best_lgb = search_lgb.best_estimator_
predict_lgb_valid = best_lgb.predict(features_valid)
rmse_lgb_valid = mean_squared_error(target_valid, predict_lgb_valid)**0.5
print('Качество модели LighGBM на валидационной выборке (корень из средней квадратичной ошибки):', rmse_lgb_valid)

Качество модели LighGBM на валидационной выборке (корень из средней квадратичной ошибки): 378.0037006957832


### XGBoost

In [14]:
model_xgb = xgb.XGBRegressor(random_state = 12345)
parameters_xgb = {'booster': ['gbtree', 'gblinear'], 
                  'max_depth':range(4, 11), 
                  'learning_rate': [0.01, 0.05, 0.1, 0.001, 0.5], 
                  'n_estimators':range(100, 501, 50)}
search_xgb = RandomizedSearchCV(model_xgb, parameters_xgb, cv = 5, n_jobs = -1, random_state = 12345)
search_xgb.fit(features_train, target_train)
best_xgb = search_xgb.best_estimator_
predict_xgb_valid = best_xgb.predict(features_valid)
rmse_xgb_valid = mean_squared_error(target_valid, predict_xgb_valid)**0.5
print('Качество модели XGBoost на валидационной выборке (корень из средней квадратичной ошибки):', rmse_xgb_valid)

[12:42:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:43:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:43:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:44:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:44:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:45:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:48:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:51:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

# 3. Анализ моделей

### CatBoost

In [15]:
%%time
start_cb_fit = time.time()
best_cb.fit(features_train, target_train, verbose = 100)
end_cb_fit = time.time()
time_cb_fit = end_cb_fit - start_cb_fit
start_cb_predict = time.time()
predict_cb_test = best_cb.predict(features_test)
end_cb_predict = time.time()
time_cb_predict = end_cb_predict - start_cb_predict
rmse_cb_test = mean_squared_error(target_test, predict_cb_test)**0.5
print('Качество модели CatBoost на тестовой выборке (корень из средней квадратичной ошибки):', rmse_cb_test)

0:	learn: 898.8298849	total: 133ms	remaining: 39.7s
100:	learn: 398.5849617	total: 22.4s	remaining: 44.1s
200:	learn: 380.5148440	total: 45.1s	remaining: 22.2s
299:	learn: 369.3954745	total: 1m 7s	remaining: 0us
Качество модели CatBoost на тестовой выборке (корень из средней квадратичной ошибки): 388.10319768211525
CPU times: user 59.8 s, sys: 8.55 s, total: 1min 8s
Wall time: 1min 12s


### LightGBM

In [16]:
%%time
start_lgb_fit = time.time()
best_lgb.fit(features_train, target_train)
end_lgb_fit = time.time()
time_lgb_fit = end_lgb_fit - start_lgb_fit
start_lgb_predict = time.time()
predict_lgb_test = best_lgb.predict(features_test)
end_lgb_predict = time.time()
time_lgb_predict = end_lgb_predict - start_lgb_predict

rmse_lgb_test = mean_squared_error(target_test, predict_lgb_test)**0.5
print('Качество модели CatBoost (корень из средней квадратичной ошибки):', rmse_lgb_test)

Качество модели CatBoost (корень из средней квадратичной ошибки): 384.4394656444861
CPU times: user 10.6 s, sys: 17.9 ms, total: 10.6 s
Wall time: 11.3 s


### XGBoost

In [17]:
%%time
start_xgb_fit = time.time()
best_xgb.fit(features_train, target_train)
end_xgb_fit = time.time()
time_xgb_fit = end_xgb_fit - start_xgb_fit
start_xgb_predict = time.time()
predict_xgb_test = best_xgb.predict(features_test)
end_xgb_predict = time.time()
time_xgb_predict = end_xgb_predict - start_xgb_predict
rmse_xgb_test = mean_squared_error(target_test, predict_xgb_test)**0.5
print('Качество модели XGBoost (корень из средней квадратичной ошибки):', rmse_xgb_test)

[13:38:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Качество модели XGBoost (корень из средней квадратичной ошибки): 377.7249628896845
CPU times: user 2min 4s, sys: 0 ns, total: 2min 4s
Wall time: 2min 9s


In [18]:
data_final = [['CatBoost', rmse_cb_test, time_cb_fit, time_cb_predict], 
              ['LightGBM', rmse_lgb_test, time_lgb_fit, time_lgb_predict], 
              ['XGBoost', rmse_xgb_test, time_xgb_fit, time_xgb_predict]]
columns_final = ['model', 'rmse','time_fit', 'time_predict']
table_final = pd.DataFrame(data = data_final, columns = columns_final)
table_final

,model,rmse,time_fit,time_predict
0,CatBoost,388.103198,72.382090,0.195629
1,LightGBM,384.439466,8.128227,3.125223
2,XGBoost,377.724963,126.155531,2.971491


# Вывод

Из полученных результатов и технического задания, можно сделать вывод, что наиболее подходящей библиотекой градиентного бустинга для решения поставленной задачи является LigthGBM, так как модель обладает близким значением RMSE к остальным моделям и наименьшим временем обучения и предсказания.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнена загрузка и подготовка данных
- [x]  Выполнено обучение моделей
- [x]  Есть анализ скорости работы и качества моделей